In [1]:
# aim: runn dnn hmm hybrid using previous code, then can condense into normal classifier structure

# get data:

import sys
sys.path.append("/Users/david/Documents/mastersCode/ubm")
print(sys.path)

['/Users/david/Documents/mastersCode/ubm/classifiers/hmm-hybrids/dnnhmm/development', '/Users/david/.conda/envs/ubm/lib/python310.zip', '/Users/david/.conda/envs/ubm/lib/python3.10', '/Users/david/.conda/envs/ubm/lib/python3.10/lib-dynload', '', '/Users/david/.conda/envs/ubm/lib/python3.10/site-packages', '/Users/david/Documents/mastersCode/ubm']


In [2]:
# test for gmm-ubm on whale calls
from importlib import reload
import audio_datastore.audio_datastore as myads
reload(myads)
from collections import Counter
import numpy as np
import config
import librosa
import utils as my_utils
import matplotlib.pyplot as plt
import feature_extraction.fe_spafe.fe_spafe as fe_spafe
import feature_extraction.fe_base as fe_base
import os.path
import dnnhmm
reload(dnnhmm)
reload(fe_base)
reload(fe_spafe)

<module 'feature_extraction.fe_spafe.fe_spafe' from '/Users/david/Documents/mastersCode/ubm/feature_extraction/fe_spafe/fe_spafe.py'>

In [3]:
# create ads
# female ads
datasetFolder = r"/Users/david/Documents/data/speech/ivectors/SPEECH DATA/FEMALE/MIC"
female_ads = myads.AudioDatastore()
female_ads.populate(datasetFolder, include_sub_folders=True, label_source=True)
Counter(female_ads.labels).values()
# male ads
datasetFolder = r"/Users/david/Documents/data/speech/ivectors/SPEECH DATA/MALE/MIC"
male_ads = myads.AudioDatastore()
male_ads.populate(datasetFolder, include_sub_folders=True, label_source=True)
Counter(male_ads.labels).values()
# combine ads
ads = myads.AudioDatastore()
ads.set([datasetFolder], files=female_ads.files + male_ads.files, labels=female_ads.labels + male_ads.labels)
# split ads
developmentLabels = ["M01", "M02", "M03", "M04", "M06", "M07", "M08", "M09", "F01", "F02", "F03", "F04", "F06", "F07",
                     "F08", "F09"]
evaluationLabels = ["M05", "M10", "F05", "F10"]
adsTrain = myads.subset(ads, developmentLabels)
adsEvaluate = myads.subset(ads, evaluationLabels)
# numFilesPerSpeakerForEnrollment = 3
adsEnroll, adsTest = myads.split(adsEvaluate, 5)
adsTest, _ = myads.split(adsTest, 5)


In [7]:
# quick aside to create pickled files to speed up things and create consistency between classifiers

from collections import Counter

print(Counter(adsEvaluate.labels).keys()) # equals to list(set(words))
print(Counter(adsEvaluate.labels).values()) # counts the elements' frequency

dict_keys(['F10', 'F05', 'M05', 'M10'])
dict_values([236, 236, 236, 236])


In [4]:
import torch
torch.set_num_threads(1)
model, torch_utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True)
(get_speech_timestamps,
 _, read_audio,
 *_) = torch_utils

Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /Users/david/.cache/torch/hub/master.zip


In [5]:
from feature_extraction.fe_configs import NormFactor
from spafe.features.mfcc import *

def get_log_features(sign, norm: NormFactor = None):
        features, _ = mel_spectrogram(sig=sign, fs=config.SAMPLE_RATE)
        features = zero_handling(features)
        features = np.log(features)
        if norm:
            features = (features - norm.means) / norm.std
            features = features - np.mean(features)
        return features

In [6]:

from audio_datastore.audio_datastore import subset
import processing.processing
import processing.process_method_base as process_method_base
import spafe.features.mfcc as mfcc
# from librosa.feature.spectral import mfcc

process_method = process_method_base.ProcessMethodBase(snr_db=None, reverb=False, model=model, get_speech_timestamps=get_speech_timestamps)

speakers = np.unique(adsEnroll.labels)
# normalisation
all_features = []
for i in range(len(speakers)):
    ads_train_subset = subset(adsEnroll, speakers[i])
    speaker_features = []
    for file in ads_train_subset.files:
        signal = process_method.pre_process(file)
        speaker_features.append(get_log_features(signal))
    speaker_features_flattened = np.array([item for sublist in speaker_features for item in sublist])
    all_features.append(speaker_features_flattened)

means = []
std = []
for feature in all_features:
    means.append(np.mean(feature, axis=0))
    std.append(np.std(feature, axis=0))

means = np.array(means)
means = np.mean(means, axis=0)

std = np.array(std)
std = np.mean(std, axis=0)

norm_factor = NormFactor(means, std)

In [7]:
from hmmlearn import hmm

speakers = np.unique(adsEnroll.labels)
speakers = speakers
hmms = {}
for i in range(len(speakers)):
    ads_train_subset = subset(adsEnroll, speakers[i])
    speaker_features = []
    for file in ads_train_subset.files:
        signal = process_method.pre_process(file)
        speaker_features.append(get_log_features(signal, norm_factor))
    speaker_features_flattened = np.array([item for sublist in speaker_features for item in sublist])
    new_hmm = hmm.GMMHMM(n_components=4, n_mix=2)
    new_hmm.fit(speaker_features_flattened)
    hmms[i] = new_hmm

In [9]:
# quick test
hmm_outer_scores = []
for i in range(len(speakers)):
    ads_train_subset = subset(adsTest, speakers[i])
    speaker_features = []
    for file in ads_train_subset.files:
        signal = process_method.pre_process(file)
        speaker_features.append(get_log_features(signal, norm_factor))
    speaker_features_flattened = np.array([item for sublist in speaker_features for item in sublist])
    hmm_inner_scores = []
    for y in range(len(hmms)):
        hmm_score = hmms[y].score(speaker_features_flattened)
        hmm_inner_scores.append(hmm_score)
    hmm_outer_scores.append(hmm_inner_scores)

# working so far !
print(np.round(hmm_outer_scores))

[[-26033. -30874. -34057. -42347.]
 [-23659. -20534. -26400. -28038.]
 [-33962. -33384. -25812. -30300.]
 [-25905. -25228. -22185. -20299.]]


In [10]:
# now add dnn
dnn_hmms = {}
for i in range(len(speakers)):
    ads_train_subset = subset(adsEnroll, speakers[i])
    speaker_features = []
    for file in ads_train_subset.files:
        signal = process_method.pre_process(file)
        speaker_features.append(get_log_features(signal, norm_factor))
    # speaker_features_flattened = np.array([item for sublist in speaker_features for item in sublist])
    new_mlp = dnnhmm.DNNHMM(hmm.GMMHMM(n_components=4, n_mix=2))
    new_mlp.train_hmm(speaker_features)
    new_mlp.train_mlp()
    dnn_hmms[i] = new_mlp

Iteration 1, loss = 0.77249282
Validation score: 0.830303
Iteration 2, loss = 0.29100970
Validation score: 0.860606
Iteration 3, loss = 0.20679184
Validation score: 0.872727
Iteration 4, loss = 0.15566818
Validation score: 0.884848
Iteration 5, loss = 0.12758465
Validation score: 0.903030
Iteration 6, loss = 0.10334486
Validation score: 0.909091
Iteration 7, loss = 0.08512648
Validation score: 0.909091
Iteration 8, loss = 0.07082514
Validation score: 0.939394
Iteration 9, loss = 0.06202437
Validation score: 0.927273
Iteration 10, loss = 0.06029941
Validation score: 0.927273
Iteration 11, loss = 0.05041202
Validation score: 0.921212
Iteration 12, loss = 0.04071344
Validation score: 0.921212
Iteration 13, loss = 0.03297901
Validation score: 0.939394
Iteration 14, loss = 0.03069996
Validation score: 0.933333
Iteration 15, loss = 0.02462052
Validation score: 0.951515
Iteration 16, loss = 0.02119992
Validation score: 0.933333
Iteration 17, loss = 0.01820187
Validation score: 0.927273
Iterat

In [11]:
# quick test
mlp_outer_scores = []
for i in range(len(speakers)):
    ads_train_subset = subset(adsTest, speakers[i])
    speaker_features = []
    for file in ads_train_subset.files:
        signal = process_method.pre_process(file)
        speaker_features.append(get_log_features(signal, norm_factor))
    speaker_features_flattened = np.array([item for sublist in speaker_features for item in sublist])
    mlp_inner_scores = []
    for y in range(len(hmms)):
        hmm_score = dnn_hmms[y].loglike_mlp(speaker_features_flattened)
        mlp_inner_scores.append(hmm_score)
    mlp_outer_scores.append(mlp_inner_scores)

# working so far !

In [27]:
print(hmm_outer_scores)

[[-26186.348405991877, -30951.582642497502, -34337.337663872735, -42511.4314143481], [-23578.60166740599, -20513.837129695086, -26463.073823870283, -28078.309609488835], [-33816.96531357519, -33439.47442495654, -25657.287939052254, -30241.137868100843], [-26215.406039149017, -25295.441801430872, -22703.556856953557, -20355.859795153203]]


In [28]:
print(mlp_outer_scores)
# doesnt really work - interesting
# maybe using rnn will work better...

[[-804.2398526071192, -816.7565163129756, -874.8330908080607, -880.2422092383745], [-710.9287883907743, -705.9644981859483, -787.6079317787905, -754.5472107983993], [-768.4360025418348, -775.4445422359298, -846.4546311481761, -803.5215430134876], [-641.3732829732586, -580.5644505148076, -680.3259458107187, -629.7522106616628]]
